In [1]:
original_text = """SESSION 11; SAFE HANDLING AND DISPOSAL OF MENSTRUAL BLOOD.

“

 

household caregiver?

2. Explain that field research in Uganda has shown that clients, caregivers, and
Peer counsellor/VHT need more support so they can know how to safely
handle and dispose of menstrual blood’. Emphasize that menstrual blood of
women with HIV does carry a risk of HIV transmission. The HIV viral load in
menstrual blood is actually higher in viral load than plasma (regular) blood
and therefore even MORE risky.

3. It is important that when handling a client’s soiled material, universal precautions are observed to reduce the risk of HIV transmission and other infectious diseases as discussed in session 10.

IMPORTANCE OF SAFE HANDLING OF MENSTRUAL BLOOD

Supplies needed to safely handle the blood often are unavailable. Many
people do not realise that female clients who are ill can still have menstru
al periods, although many who are severely ill often do stop menstruating.
Now that many women are taking the ARV AIDS drugs, even women who had
previously stopped menstruating are now returning to their menses. Most
women do not like to talk about their menstrual periods and are unclear that
HIV (and other illnesses/infections) can be transmitted to others by unsafe
exposure to menstrual blood. Female clients who may be very sick and/or
bed bound require sensitive and practical care during their menstrual periods
from household caregivers and Peer counsellor/VHT. It is essential to provide
care in a way that helps the female client maintain her dignity so she can feel
confident and in-control of managing her menstrual period.

C. Materials that Can Be Used for Menstrual Periods (10 minutes)

1. Ask participants, What products or materials do women use to help soak up
menstrual blood and keep female clients clean? Record responses on the
flipchart.

2. Display the different types materials that are commonly used for soaking
menstrual blood.

3. Pass out and review with the group, How to Make a Re-usable Pad. Clarify
any questions participants may have.

4. Explain to participants that menstrual blood of HIV-positive female clients
can contain the HIV virus. However, there is an extremely low risk of getting
HIV through care giving activities if one follows universal precautions (such

7 Xavier Nsabagasani and Brendon Barnes. 2008. Testing Small Doable Actions to Improve Hygiene Practices in

the Care of People Living With HIV/AIDS. Hygiene Improvement Project. Plan Uganda; and, Xavier Nsabagasani 79
and Brendon Barnes. 2008. Identifying Small Doable Actions to Improve Hygiene Practices in the Care of

People Living With HIV/AIDS: Focus Group Discussions and In-Depth Interviews. Hygiene Improvement Project.

Plan Uganda.
"""

In [2]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

long_text = original_text #"This is a very very long text. " * 300

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# tokenize without truncation
inputs_no_trunc = tokenizer(long_text, max_length=None, return_tensors='pt', truncation=False)

# get batches of tokens corresponding to the exact model_max_length
chunk_start = 0
chunk_end = tokenizer.model_max_length  # == 1024 for Bart
inputs_batch_lst = []
while chunk_start <= len(inputs_no_trunc['input_ids'][0]):
    inputs_batch = inputs_no_trunc['input_ids'][0][chunk_start:chunk_end]  # get batch of n tokens
    inputs_batch = torch.unsqueeze(inputs_batch, 0)
    inputs_batch_lst.append(inputs_batch)
    chunk_start += tokenizer.model_max_length  # == 1024 for Bart
    chunk_end += tokenizer.model_max_length  # == 1024 for Bart

# generate a summary on each batch
summary_ids_lst = [model.generate(inputs, num_beams=4, max_length=200, early_stopping=True) for inputs in inputs_batch_lst]

# decode the output and join into one string with one paragraph per summary batch
summary_batch_lst = []
for summary_id in summary_ids_lst:
    summary_batch = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_id]
    summary_batch_lst.append(summary_batch[0])
summary_all = '\n'.join(summary_batch_lst)

print(summary_all.replace(".", ".\n"))

Downloading:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Menstrual blood of HIV-positive female clients can contain the HIV virus.
 There is an extremely low risk of getting HIV through care giving activities if one follows universal precautions (such as using a re-usable pad) Many people do not realise that female clients who are ill can still have menstrual periods.



In [4]:
import pandas as pd
import os
import torch
from transformers import LongformerTokenizer, EncoderDecoderModel
from transformers import BartTokenizer, BartForConditionalGeneration

long_text = original_text
device = "cuda:1" if torch.cuda.is_available() else "cpu"
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
inputs_no_trunc = tokenizer(long_text, max_length=None, return_tensors='pt', truncation=False).to(device)
model     = model.to(device)
chunk_start = 0
chunk_end = tokenizer.model_max_length  # == 1024 for Bart
inputs_batch_lst = []
while chunk_start <= len(inputs_no_trunc['input_ids'][0]):
    inputs_batch = inputs_no_trunc['input_ids'][0][chunk_start:chunk_end]  # get batch of n tokens
    inputs_batch = torch.unsqueeze(inputs_batch, 0)
    inputs_batch_lst.append(inputs_batch)
    chunk_start += tokenizer.model_max_length  # == 1024 for Bart
    chunk_end += tokenizer.model_max_length  # == 1024 for Bart
summary_ids_lst = [model.generate(inputs, num_beams=4, max_length=200, early_stopping=True) for inputs in inputs_batch_lst]
summary_batch_lst = []
for summary_id in summary_ids_lst:
    summary_batch = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_id]
    summary_batch_lst.append(summary_batch[0])
summary_all = '\n'.join(summary_batch_lst)

print(summary_all.replace(".", ".\n"))

Menstrual blood of HIV-positive female clients can contain the HIV virus.
 There is an extremely low risk of getting HIV through care giving activities if one follows universal precautions (such as using a re-usable pad) Many people do not realise that female clients who are ill can still have menstrual periods.



In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("valhalla/longformer-base-4096-finetuned-squadv1")

# Initialize the model
model = AutoModelForQuestionAnswering.from_pretrained("valhalla/longformer-base-4096-finetuned-squadv1")

# Specify text and question
text = """Germany (German: Deutschland, German pronunciation: [ˈdɔʏtʃlant]), officially the Federal Republic of Germany,[e] is a country at the intersection of Central and Western Europe. It is situated between the Baltic and North seas to the north, and the Alps to the south; covering an area of 357,022 square kilometres (137,847 sq mi), with a population of over 83 million within its 16 constituent states. It borders Denmark to the north, Poland and the Czech Republic to the east, Austria and Switzerland to the south, and France, Luxembourg, Belgium, and the Netherlands to the west. Germany is the second-most populous country in Europe after Russia, as well as the most populous member state of the European Union. Its capital and largest city is Berlin, and its financial centre is Frankfurt; the largest urban area is the Ruhr.Various Germanic tribes have inhabited the northern parts of modern Germany since classical antiquity. A region named Germania was documented before AD 100. In the 10th century, German territories formed a central part of the Holy Roman Empire. During the 16th century, northern German regions became the centre of the Protestant Reformation. Following the Napoleonic Wars and the dissolution of the Holy Roman Empire in 1806, the German Confederation was formed in 1815. In 1871, Germany became a nation-state when most of the German states unified into the Prussian-dominated German Empire. After World War I and the German Revolution of 1918–1919, the Empire was replaced by the semi-presidential Weimar Republic. The Nazi seizure of power in 1933 led to the establishment of a dictatorship, World War II, and the Holocaust. After the end of World War II in Europe and a period of Allied occupation, Germany was divided into the Federal Republic of Germany, generally known as West Germany, and the German Democratic Republic, East Germany. The Federal Republic of Germany was a founding member of the European Economic Community and the European Union, while the German Democratic Republic was a communist Eastern Bloc state and member of the Warsaw Pact. After the fall of communism, German reunification saw the former East German states join the Federal Republic of Germany on 3 October 1990—becoming a federal parliamentary republic led by a chancellor.Germany is a great power with a strong economy; it has the largest economy in Europe, the world's fourth-largest economy by nominal GDP, and the fifth-largest by PPP. As a global leader in several industrial, scientific and technological sectors, it is both the world's third-largest exporter and importer of goods. As a developed country, which ranks very high on the Human Development Index, it offers social security and a universal health care system, environmental protections, and a tuition-free university education. Germany is also a member of the United Nations, NATO, the G7, the G20, and the OECD. It also has the fourth-greatest number of UNESCO World Heritage Sites."""
question = "How large is Germany's economy by nominal GDP?"

# Tokenize the input text
encoding = tokenizer(question, text, return_tensors="pt")
input_ids = encoding["input_ids"]

# Get attention mask (local + global attention)
attention_mask = encoding["attention_mask"]

# Get the predictions
start_scores, end_scores = model(input_ids, attention_mask=attention_mask).values()

# Convert predictions into answer
all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())
answer_tokens = all_tokens[torch.argmax(start_scores) :torch.argmax(end_scores)+1]
answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))

# Print answer
print(answer)

Downloading:   0%|          | 0.00/757 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/595M [00:00<?, ?B/s]

 fourth-largest


In [3]:
from transformers import LongformerTokenizer, EncoderDecoderModel

# Load model and tokenizer
model = EncoderDecoderModel.from_pretrained("patrickvonplaten/longformer2roberta-cnn_dailymail-fp16")
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096") 

# Specify the article
article = """Germany (German: Deutschland, German pronunciation: [ˈdɔʏtʃlant]), officially the Federal Republic of Germany,[e] is a country at the intersection of Central and Western Europe. It is situated between the Baltic and North seas to the north, and the Alps to the south; covering an area of 357,022 square kilometres (137,847 sq mi), with a population of over 83 million within its 16 constituent states. It borders Denmark to the north, Poland and the Czech Republic to the east, Austria and Switzerland to the south, and France, Luxembourg, Belgium, and the Netherlands to the west. Germany is the second-most populous country in Europe after Russia, as well as the most populous member state of the European Union. Its capital and largest city is Berlin, and its financial centre is Frankfurt; the largest urban area is the Ruhr.Various Germanic tribes have inhabited the northern parts of modern Germany since classical antiquity. A region named Germania was documented before AD 100. In the 10th century, German territories formed a central part of the Holy Roman Empire. During the 16th century, northern German regions became the centre of the Protestant Reformation. Following the Napoleonic Wars and the dissolution of the Holy Roman Empire in 1806, the German Confederation was formed in 1815. In 1871, Germany became a nation-state when most of the German states unified into the Prussian-dominated German Empire. After World War I and the German Revolution of 1918–1919, the Empire was replaced by the semi-presidential Weimar Republic. The Nazi seizure of power in 1933 led to the establishment of a dictatorship, World War II, and the Holocaust. After the end of World War II in Europe and a period of Allied occupation, Germany was divided into the Federal Republic of Germany, generally known as West Germany, and the German Democratic Republic, East Germany. The Federal Republic of Germany was a founding member of the European Economic Community and the European Union, while the German Democratic Republic was a communist Eastern Bloc state and member of the Warsaw Pact. After the fall of communism, German reunification saw the former East German states join the Federal Republic of Germany on 3 October 1990—becoming a federal parliamentary republic led by a chancellor.Germany is a great power with a strong economy; it has the largest economy in Europe, the world's fourth-largest economy by nominal GDP, and the fifth-largest by PPP. As a global leader in several industrial, scientific and technological sectors, it is both the world's third-largest exporter and importer of goods. As a developed country, which ranks very high on the Human Development Index, it offers social security and a universal health care system, environmental protections, and a tuition-free university education. Germany is also a member of the United Nations, NATO, the G7, the G20, and the OECD. It also has the fourth-greatest number of UNESCO World Heritage Sites."""

# Tokenize and summarize
input_ids = tokenizer(article, return_tensors="pt").input_ids
output_ids = model.generate(input_ids)

# Get the summary from the output tokens
summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print summary
print(summary.replace(".", ".\n"))

Downloading:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

Some weights of the model checkpoint at patrickvonplaten/longformer2roberta-cnn_dailymail-fp16 were not used when initializing EncoderDecoderModel: ['decoder.roberta.pooler.dense.weight', 'decoder.roberta.pooler.dense.bias']
- This IS expected if you are initializing EncoderDecoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EncoderDecoderModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Germany is the second-most populous country in Europe after Russia.
It is the country's second-largest economy and the most populous member state of the European Union.
Germany is also a member of the United Nations, the G7, the OECD and the G20.


In [3]:
from transformers import LongformerTokenizer, EncoderDecoderModel

# Load model and tokenizer
model = EncoderDecoderModel.from_pretrained("patrickvonplaten/longformer2roberta-cnn_dailymail-fp16")
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096") 

# Specify the article
article = original_text
# Tokenize and summarize
input_ids = tokenizer(article, return_tensors="pt").input_ids
output_ids = model.generate(input_ids)

# Get the summary from the output tokens
summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print summary
print(summary.replace(".", ".\n"))

Some weights of the model checkpoint at patrickvonplaten/longformer2roberta-cnn_dailymail-fp16 were not used when initializing EncoderDecoderModel: ['decoder.roberta.pooler.dense.weight', 'decoder.roberta.pooler.dense.bias']
- This IS expected if you are initializing EncoderDecoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EncoderDecoderModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


HIV virus carries a risk of HIV transmission.

Use these precautions to help women retain their dignity.

The HIV viral load in women women with HIV carries a.

risk of HIV.

Most women do not like to talk about their periods.

Many who are ill can still have menstruational periods.



In [1]:
import pandas as pd
import os
import torch
from transformers import LongformerTokenizer, EncoderDecoderModel
from transformers import BartTokenizer, BartForConditionalGeneration

In [8]:
folder_to_save = "../tmp/parsed_pdfs_longformer"
folder_to_process = "../tmp/parsed_pdfs"
model = EncoderDecoderModel.from_pretrained("patrickvonplaten/longformer2roberta-cnn_dailymail-fp16")
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
device = "cuda:1" if torch.cuda.is_available() else "cpu"
os.makedirs(folder_to_save, exist_ok=True)
for idx, file in enumerate(os.listdir(folder_to_process)):
    if idx % 20 == 0:
        print(idx)
    df = pd.read_excel(os.path.join(folder_to_process, file), engine='openpyxl').fillna("")
    if os.path.exists(os.path.join(folder_to_save, file)):
        print("Already processed ", file)
        continue
    else:
        print(file)
    df["summary"] = ""
    for i in range(len(df)):
        try:
            # Tokenize and summarize
            inputs_no_trunc = tokenizer(df["Text"].values[i], max_length=None, return_tensors='pt', truncation=False).to(device)
            model     = model.to(device)
            chunk_start = 0
            chunk_end = tokenizer.model_max_length  # == 1024 for Bart
            inputs_batch_lst = []
            while chunk_start <= len(inputs_no_trunc['input_ids'][0]):
                inputs_batch = inputs_no_trunc['input_ids'][0][chunk_start:chunk_end]  # get batch of n tokens
                inputs_batch = torch.unsqueeze(inputs_batch, 0)
                inputs_batch_lst.append(inputs_batch)
                chunk_start += tokenizer.model_max_length  # == 1024 for Bart
                chunk_end += tokenizer.model_max_length  # == 1024 for Bart
            summary_ids_lst = [model.generate(inputs, num_beams=4, max_length=200, early_stopping=True) for inputs in inputs_batch_lst]
            summary_batch_lst = []
            for summary_id in summary_ids_lst:
                summary_batch = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_id]
                summary_batch_lst.append(summary_batch[0])
            summary_all = '\n'.join(summary_batch_lst)
            df["summary"].values[i] = summary_all
        except:
            pass
    df.to_excel(os.path.join(folder_to_save, file))

Some weights of the model checkpoint at patrickvonplaten/longformer2roberta-cnn_dailymail-fp16 were not used when initializing EncoderDecoderModel: ['decoder.roberta.pooler.dense.weight', 'decoder.roberta.pooler.dense.bias']
- This IS expected if you are initializing EncoderDecoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EncoderDecoderModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0
Already processed  PA00KNS3.xlsx
PNABN631.xlsx


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

PA00KR9N.xlsx


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

AID-623-BC-11-00001.xlsx


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

PDAAX663.xlsx


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [7]:
folder_to_save = "../tmp/parsed_pdfs_bart"
folder_to_process = "../tmp/parsed_pdfs"
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
device = "cuda:1" if torch.cuda.is_available() else "cpu"
os.makedirs(folder_to_save, exist_ok=True)
for idx, file in enumerate(os.listdir(folder_to_process)):
    if idx % 20 == 0:
        print(idx)
    df = pd.read_excel(os.path.join(folder_to_process, file), engine='openpyxl').fillna("")
    if os.path.exists(os.path.join(folder_to_save, file)):
        print("Already processed ", file)
        continue
    else:
        print(file)
    df["summary"] = ""
    for i in range(len(df)):
        try:
            # Tokenize and summarize
            inputs_no_trunc = tokenizer(df["Text"].values[i], max_length=None, return_tensors='pt', truncation=False).to(device)
            model     = model.to(device)
            chunk_start = 0
            chunk_end = tokenizer.model_max_length  # == 1024 for Bart
            inputs_batch_lst = []
            while chunk_start <= len(inputs_no_trunc['input_ids'][0]):
                inputs_batch = inputs_no_trunc['input_ids'][0][chunk_start:chunk_end]  # get batch of n tokens
                inputs_batch = torch.unsqueeze(inputs_batch, 0)
                inputs_batch_lst.append(inputs_batch)
                chunk_start += tokenizer.model_max_length  # == 1024 for Bart
                chunk_end += tokenizer.model_max_length  # == 1024 for Bart
            summary_ids_lst = [model.generate(inputs, num_beams=4, max_length=200, early_stopping=True) for inputs in inputs_batch_lst]
            summary_batch_lst = []
            for summary_id in summary_ids_lst:
                summary_batch = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_id]
                summary_batch_lst.append(summary_batch[0])
            summary_all = '\n'.join(summary_batch_lst)
            df["summary"].values[i] = summary_all
        except:
            pass
    df.to_excel(os.path.join(folder_to_save, file))

0
Already processed  PA00KNS3.xlsx
Already processed  PNABN631.xlsx
Already processed  PA00KR9N.xlsx
Already processed  PA00M6CZ.xlsx
Already processed  PA00KPK2.xlsx
Already processed  PDAAP519.xlsx
Already processed  PA00N69H.xlsx
Already processed  278-C-17-00002.xlsx
Already processed  PA00MQ6F.xlsx
Already processed  PA00JQT3.xlsx
Already processed  PA00N4J4.xlsx
Already processed  PA00TK92.xlsx
Already processed  PA00KK9D.xlsx
Already processed  PA00T5HB.xlsx
Already processed  PA00N4Q2.xlsx
Already processed  PA00N4PN.xlsx
Already processed  PA00TRVX.xlsx
Already processed  PDAAC044D1.xlsx
Already processed  PNAAV944.xlsx
Already processed  PA00T155.xlsx
20
Already processed  PA00WH9Z.xlsx
Already processed  PDACN883.xlsx
Already processed  PA00KC72.xlsx
Already processed  PNAAC849.xlsx
Already processed  PA00SV1V.xlsx
Already processed  PNAAU412.xlsx
Already processed  PA00SRVF.xlsx
Already processed  PNADZ994.xlsx
Already processed  PBAAA335.xlsx
Already processed  PA00TK8Z.xl

Token indices sequence length is longer than the specified maximum sequence length for this model (1389 > 1024). Running this sequence through the model will result in indexing errors


PA00SXZR.xlsx
PA00K8RF.xlsx
AID-621-TO-16-00003.xlsx
PA00TD2Z.xlsx
PA00N4Z5.xlsx
AID-521-C-17-00002.xlsx
PDAAA191B1.xlsx
PNAEA784.xlsx
PA00TWK6.xlsx
PDACR573.xlsx
100
PBAAE636.xlsx
PNADR651.xlsx
PNAAV254.xlsx
PA00W76F.xlsx
PNAAY001.xlsx
PDAAT718.xlsx
PA00T5XB.xlsx
PA00MPDR.xlsx
PA00JX4V.xlsx
PBAAF117.xlsx
PA00K8R5.xlsx
PDAAS594.xlsx
PA00WC1W.xlsx
PA00KXN2.xlsx
PDABD541.xlsx
PDAAU331.xlsx
PA00J97Z.xlsx
PNAAN748.xlsx
AID-517-A-16-00001.xlsx
PA00MPQT.xlsx
120
PDAAV603.xlsx
PA00WS5S.xlsx
PA00MMVK.xlsx
PNAED176.xlsx
PDAAD963B1.xlsx
PA00MP69.xlsx
PA00JRPF.xlsx
PDAAU348.xlsx
PDACQ818.xlsx
PDACX620.xlsx
PA00MPBG.xlsx
PA00WKN1.xlsx
PA00MM17.xlsx
PA00TFQD.xlsx
PA00TT8F.xlsx
PDABJ629.xlsx
PA00K9ZB.xlsx
AID-278-A-17-00002.xlsx
PA00SXZP.xlsx
PDACW232.xlsx
140
PA00KM2V.xlsx
PA00JMR8.xlsx
PDAAT593.xlsx
PA00JPRX.xlsx
PA00TRM3.xlsx
PA00JZ5T.xlsx
PBAAA571.xlsx
PA00JRPM.xlsx
PNAAR356.xlsx
PDACO418.xlsx
PA00TRW3.xlsx
PA00T3P1.xlsx
PA00MZ5D.xlsx
PBAAD339.xlsx
PA00JR3H.xlsx
PDACP848.xlsx
PA00JX4J.xlsx
PDACS

PA00N4N5.xlsx
PBAAA219.xlsx
PA00TFQR.xlsx
PA00TDHM.xlsx
PA00SZ11.xlsx
PDAAT715.xlsx
640
AID-612-TO-16-00004.xlsx
PA00KT6F.xlsx
PA00JBSW.xlsx
PA00MBVN.xlsx
PDAAV474.xlsx
PA00KNBJ.xlsx
PA00TZG1.xlsx
PDABF642.xlsx
PA00N4NP.xlsx
PA00N4P9.xlsx
PA00KKKM.xlsx
PA00MV8P.xlsx
PA00MC4D.xlsx
PNAAW701.xlsx
PA00JX4S.xlsx
PA00TPWQ.xlsx
PNAAL471.xlsx
PA00TFD7.xlsx
PDACR854.xlsx
AID-268-N-15-00001.xlsx
660
PA00T94D.xlsx
PA00M4MC.xlsx
PA00KNFP.xlsx
PA00W6H5.xlsx
PNAAV299.xlsx
PA00TDNF.xlsx
PDABH144.xlsx
PNADP825.xlsx
PDABF389.xlsx
PA00JHZT.xlsx
PDACM484.xlsx
PBAAE370.xlsx
PNAAU624.xlsx
PNAAX035.xlsx
PA00D5T2.xlsx
PA00N4RW.xlsx
PA00X2J2.xlsx
PNAEB629.xlsx
AID-617-A-15-00010.xlsx
PA00KMCP.xlsx
680
PA00KG3Q.xlsx
PA00JHGG.xlsx
PNAAN332.xlsx
PA00JX5Q.xlsx
PA00N7P3.xlsx
PBAAE363.xlsx
PDACU927.xlsx
PA00TRV2.xlsx
PA00MV8Z.xlsx
PA00SRV5.xlsx
PA00WH24.xlsx
PA00THN8.xlsx
PA00SSG4.xlsx
AID-612-A-16-00003.xlsx
669-A-00-09-00001-00.xlsx
PA00THS1.xlsx
PA00TQGJ.xlsx
PA00JVM6.xlsx
PNADT198.xlsx
PA00MMXX.xlsx
700
PDACY32